This notebook was inspired by [this](https://towardsdatascience.com/turning-non-image-data-into-images-for-classification-is-surprisingly-effective-70ce82cfee27) article by [Andre Ye](https://www.linkedin.com/in/andre-ye).


Model training/code comes from [here](https://github.com/alok-ai-lab/DeepInsight/blob/master/examples/pytorch_squeezenet.ipynb). Thanks to [Keith A. Boroevich](https://github.com/kaboroevich)/kaboroevich for the starter code.


This is my first dive into CNNs, so this model is pretty sketchy and it model seems to be very poor. In fact, if I pick even samples of each class all probabilities are equal to 1/number of classes. I think I've done something wrong here, or this is just a inappropriate use of this approach.

In [ ]:
#pytorch utils
import torch
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

#check gpu
print(torch.cuda.is_available())
device = torch.device(device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

#deep insight utils
!pip install git+git://github.com/alok-ai-lab/DeepInsight.git#egg=DeepInsight
from pyDeepInsight import ImageTransformer, LogScaler

#sklearn utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.metrics import log_loss

#essential utils
import pandas as pd
import numpy as np

#read in data and get sample since it is too big to handle otherwise
training = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")
training = training.sample(frac = .25)

In [ ]:
#load untrained model
from torchvision.models import squeezenet1_1
net = squeezenet1_1(pretrained=False).float()
net.classifier[1] = nn.Conv2d(512, 9, kernel_size=(1,1), stride=(1,1)).to(device)

#create tnse
tsne = TSNE(
    n_components = 2,
    random_state = 1,
    n_jobs=-1)

#create transformer
it = ImageTransformer(
    feature_extractor=tsne, 
    pixels = 18)

#create scaler
ls = LogScaler()
ls.fit(np.ascontiguousarray(training.iloc[:,1:76]))

#create tensor transformer
preprocess = transforms.Compose([transforms.ToTensor()])

In [ ]:
def CNNPIPE(model, data, batch = 32, epochs = 20, learning = .01, momentum = 0.9, reg = 0):    
    
    #make sure model is on device
    net = model.to(device)
    
    results = {}
    
    #pull out features
    X = data.iloc[:, 1:76]
    X = np.ascontiguousarray(X)

    #create label encoder
    le = LabelEncoder()

    #pull out responses
    y = data.iloc[:, 76]
    y = le.fit_transform(y)

    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1, stratify = y)
    
    #transform
    ls = LogScaler()
    X_train = ls.fit_transform(X_train)
    X_test = ls.transform(X_test)

    #create matricies
    X_train = it.fit_transform(X_train)
    X_test = it.transform(X_test)
    
    #create tensors
    X_train = torch.stack([preprocess(img) for img in X_train]).to(device)
    y_train = torch.from_numpy(le.fit_transform(y_train)).to(device)
    X_test = torch.stack([preprocess(np.float32(img)) for img in X_test]).to(device, dtype = torch.float)
    
    #load data
    trainset = TensorDataset(X_train, y_train)
    trainloader = DataLoader(trainset, batch_size = batch, shuffle = True)
    
    #set loss function and add to device
    criterion = nn.CrossEntropyLoss()
    criterion.to(device)
    optimizer = optim.SGD(net.parameters(), lr = learning, momentum = momentum, weight_decay = reg)

    #run epochs
    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
       
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            #update loss
            running_loss += loss.item()
        
        #average loss
        train_score = round(running_loss / len(X_train) * batch, 4)
        
        #get test predictions
        test_outputs = net(X_test)
        preds = torch.nn.functional.softmax(test_outputs, 1)
        preds = preds.to('cpu')
        preds = pd.DataFrame(preds.detach().numpy())
        
        #score predictions
        test_score = round(log_loss(y_test, preds), 4)
        
        #results to dict
        results[epoch] = {'train_score' : train_score, 'test_score': test_score}
        print("Epoch: ", epoch, " Train score: ", train_score, " Test score: ", test_score)
    
    return results

In [ ]:
#run
results = CNNPIPE(net, training, batch = 32, epochs = 15, learning = .1, reg = .01)
print(results)

The part below makes the test predictions.

In [ ]:
#load data
raw_test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
raw_train = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")

#grab ids
ids = raw_test.iloc[:,0]

#grab features
training = raw_train.iloc[:,1:76]
training = np.ascontiguousarray(training)
test = raw_test.iloc[:,1:76]
test = np.ascontiguousarray(test)

#scale
ls = LogScaler()
ls.fit(training)

In [ ]:
#create tnse
tsne = TSNE(n_components = 2, random_state = 1, n_jobs=-1)

#create transformer
it = ImageTransformer(feature_extractor = tsne, pixels = 75)

#fit transformer images
train = it.fit(training)

In [ ]:
#create prep
preprocess = transforms.Compose([transforms.ToTensor()])

def predict(obs):
    
    #preprocess
    obs = obs.reshape(1, -1)
    obs = ls.transform(obs)
    obs = it.transform(obs)
    obs = torch.stack([preprocess(np.float32(img)) for img in obs]).to(device, dtype = torch.float)
    
    #infer
    outs = net(obs)
    outs = torch.nn.functional.softmax(outs, 1)
    outs = outs.cpu()
    outs = outs.detach().numpy()
    outs = outs.tolist()
    outs = [item for sublist in outs for item in sublist]
    return outs

#save inference
rep = []
for row in test:
    rep.append(predict(row))

In [ ]:
#make submission
ids = pd.DataFrame(ids, columns = ['id'])

rep = pd.DataFrame(rep)

submission = ids.merge(rep, left_index = True, right_index = True)

submission.columns = ["id", "Class_1", "Class_2", "Class_3", "Class_4",
                  "Class_5", "Class_6", "Class_7", "Class_8", "Class_9"]

submission.to_csv('submission.csv', index = False)